In [1]:
# import modules
import pandas as pd
import numpy as np
import spotipy

# full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas format
pd.set_option('display.float_format', '{:_.0f}'.format)
# NOTE: underscore separaters ('_') are better than commas (',') because 
# numbers with underscores work in Python without any extra effort.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Can we find Metadata for The Billboard 100?

In [2]:
# Billboard Top 100 Historical Data
# link:  https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs
# via:  https://toolbox.google.com/datasetsearch
url_billboard = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'

In [3]:
df_billboard = pd.read_csv(url_billboard)
df_billboard['date'] = pd.to_datetime(df_billboard['date'])
df_billboard.head(1)

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1,1,3


In [4]:
# date range for data
df_billboard.date.min(), df_billboard.date.max()

(Timestamp('1958-08-04 00:00:00'), Timestamp('2021-11-06 00:00:00'))

In [5]:
# just the songs on the billboard 100
df_billboard_songs = df_billboard[['song', 'artist']].drop_duplicates().sort_values(['artist', 'song']).reset_index(drop=True)
df_billboard_songs.shape

(29681, 2)

In [6]:
df_billboard_songs.sample(10)

,song,artist
25820,Who'll Be The Next In Line,The Kinks
12720,Big Boss Man,Jimmy Reed
26721,Snoopy For President,The Royal Guardsmen
7328,Roland The Roadie And Gertrude The Groupie,Dr. Hook And The Medicine Show
28593,Easy Livin,Uriah Heep
1533,When You're Gone,Avril Lavigne
27362,A Lonely Night,The Weeknd
4166,You Make Me Work,Cameo
22812,Black Pearl,"Sonny Charles And The Checkmates, Ltd."
18118,The Search,NF


In [7]:
# there are ~30k songs on the Billboard 100 list
# let's see how many are here:
# https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs
# via:  https://www.kaggle.com/datasets/
# Spotify 1.2M+ Songs
url_1M_songs = r'D:\RYERSON\820\Datasets\Spotify 1.2M+ Songs\tracks_features.csv'

df_1M_songs = pd.read_csv(url_1M_songs)

In [ ]:
# many indirect matches, need be clever to merge
# match artist, look for billboard.song in 1M.name.str

In [8]:
# change merging fields to lowercase with matching names
df_billboard_songs['artist_lower'] = df_billboard_songs['artist'].str.lower()
df_billboard_songs['track_lower'] = df_billboard_songs['song'].str.lower()

df_billboard_songs = df_billboard_songs[['artist_lower', 'track_lower']]

In [9]:
df_1M_songs['artists_lower'] = df_1M_songs['artists'].str.lower()
df_1M_songs['track_lower'] = df_1M_songs['name'].str.lower()

In [27]:
# now check if df_billboard_songs entries match 1M_songs entries

billboard_tracks = list(df_billboard_songs.track_lower)

# df_billboard_songs['ID'] = df_1M_songs[(df_billboard_songs.track_lower in df_1M_songs.track_lower) & (df_billboard_songs.artist_lower in df_1M_songs.artists_lower)]


In [24]:
df_1M_songs.head(0)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artists_lower,track_lower


In [47]:
df_temp = df_billboard_songs.sample(100)

In [48]:
df_temp2 = df_temp.copy()

for index, row in df_temp.iterrows():
    # print(row['artist_lower'], row['track_lower'])
    song = row['track_lower']
    temp = df_1M_songs[song in df_1M_songs.track_lower]
    display(temp)
        

KeyError: False

### regex method only takes first artist, doesn't work properly misses out of order artists

In [ ]:
# formatting in df_1M_songs is difficult to parse
# need to extract list-style formatting using Regex:
"""
https://stackoverflow.com/questions/71469808/how-to-replace-a-list-with-first-element-of-list-in-pandas-dataframe-column
regex:
(             # start capturing
(?<=\[["\'])  # if preceded by [" or ['
[^"\']*       # get all text until " or '
|             # OR
^[^"\']+$     # get whole string if it doesn't contain " or '
)             # stop capturing
"""
df_1M_songs['artist_lower'] = df_1M_songs['artists'].str.extract('((?<=\[["\'])[^"\']*|^[^"\']+$)')
df_1M_songs['artist_lower'] = df_1M_songs['artist_lower'].str.lower()
df_1M_songs['track_lower'] = df_1M_songs['name'].str.lower()

In [ ]:
# now, let's see how many id matches we can get with the billboard tracks
merge_on = ['artist_lower']

df_temp = pd.merge(df_billboard_songs, df_1M_songs, how='left', on=merge_on, indicator=True)
df_temp.shape

In [ ]:
df_temp.head()

In [ ]:
df_1M_songs[df_1M_songs.artist_lower.str.contains('yankovic')].shape

In [ ]:
df_1M_songs[df_1M_songs.artists.str.contains('Yankovic')].shape

In [ ]:
# merge billboard 100 with 1.2M songs

# change merging fields to lowercase with matching names
df_billboard['artist_lower'] = df_billboard['artist'].str.lower()
df_billboard['track_lower'] = df_billboard['song'].str.lower()

# need to extract list-style formatting using Regex:
"""
https://stackoverflow.com/questions/71469808/how-to-replace-a-list-with-first-element-of-list-in-pandas-dataframe-column
regex:
(             # start capturing
(?<=\[["\'])  # if preceded by [" or ['
[^"\']*       # get all text until " or '
|             # OR
^[^"\']+$     # get whole string if it doesn't contain " or '
)             # stop capturing
"""
df_1M_songs['artist_lower'] = df_1M_songs['artists'].str.extract('((?<=\[["\'])[^"\']*|^[^"\']+$)')
df_1M_songs['artist_lower'] = df_1M_songs['artist_lower'].str.lower()
df_1M_songs['track_lower'] = df_1M_songs['name'].str.lower()

# join tables
df = pd.merge(df_billboard, df_1M_songs, how='outer', on=['artist_lower', 'track_lower'], indicator=True)
df.shape, df[df._merge=='both'].shape

In [ ]:
# how many missing songs?
df[df._merge == 'left_only'].drop_duplicates(['artist_lower', 'track_lower']).reset_index().shape

In [ ]:
# how many found songs?
df[df._merge == 'both'].drop_duplicates(['artist_lower', 'track_lower']).reset_index().shape
# lame

In [ ]:
missing_songs = df[df._merge == 'left_only'].drop_duplicates(['artist_lower', 'track_lower']).sort_values(['artist_lower', 'track_lower']).reset_index()[['artist_lower', 'track_lower']]
missing_songs

In [ ]:
# manually check database for these
manual_check = df_1M_songs[['artist_lower', 'track_lower', 'album']]

In [ ]:
manual_check[manual_check.track_lower == 'velcro fly']

In [ ]:
# can i use this to join?
manual_check[manual_check.track_lower.str.contains('velcro fly')]

# don't like this solution, it will be humongous
# https://stackoverflow.com/questions/58011182/pandas-merge-a-dataframe-on-an-exact-and-partial-match

In [ ]:
manual_check[manual_check.artist_lower=='zz top'].sort_values('track_lower')
# they're all in there, maybe we need a partial match for the merge?

In [ ]:
# which songs were not matched to billboard 100?
unmatched = df[df._merge == 'right_only'].drop_duplicates(['artist_lower', 'track_lower'])[['artists', 'artist_lower', 'track_lower']].sort_values(['artist_lower', 'track_lower']).reset_index()

In [ ]:
unmatched.sample(20)

### Can we merge Billboard 100 with Song Metadata?
### not well (~90% missing)
### NLP section only
    * we could use this merge for NLP
    * this could be a representative sample
    * discard for now because it would be nice to use the entire Billboard 100 dataset

In [ ]:
# Billboard Top 100 Historical Data
# link:  https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs
# via:  https://toolbox.google.com/datasetsearch
url_billboard = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'

# Music Dataset: Lyrics and Metadata from 1950 to 2019
# link:  https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs
# via:  https://toolbox.google.com/datasetsearch
# info re metadata:  https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
# also available via Spotify web API:
    # link:  https://developer.spotify.com/documentation/web-api/
url_songdata = r'D:\RYERSON\820\Datasets\Music Dataset Lyrics and Metadata from 1950 to 2019\tcc_ceds_music.csv'


In [ ]:
df_billboard = pd.read_csv(url_billboard)

# lowercase for joining datasets
# these match the names in the other dataframe, and are lowercase, as the other dataset is
df_billboard['artist_name'] = df_billboard['artist'].str.lower()
df_billboard['track_name'] = df_billboard['song'].str.lower()

df_billboard.head(1)

In [ ]:
df_songdata = pd.read_csv(url_songdata)
# df_songdata.rename(columns={'artist_name': 'artist', 'track_name': 'song'}, inplace=True)
df_songdata.head(1)

In [ ]:
df_billboard.shape, df_songdata.shape

In [ ]:
df = pd.merge(df_billboard, df_songdata, how='left', on=['artist_name', 'track_name'], indicator=True)
df.shape, df[df._merge=='both'].shape

In [ ]:
# lots of missing hits, let's check what's up
temp_columns = ['artist_name', 'track_name', '_merge']
temp = df[temp_columns]

In [ ]:
temp.sample(10)

In [ ]:
df_songdata[df_songdata.artist_name=='vikki carr']

In [ ]:
df_songdata[df_songdata.artist_name=='walk the moon']

In [ ]:
# ok, how many songs are missing? should we use the spotify API instead?

# missing tracks
missing = temp[temp._merge=='left_only'].drop_duplicates(keep='first')

# total tracks
all_tracks = df_billboard[['song', 'artist']].drop_duplicates(keep='first')

In [ ]:
missing.shape[0] / all_tracks.shape[0]
# 90% of songs are missing